In [1]:
import os
import cv2
import keras
import numpy as np
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, Dropout, MaxPooling2D, UpSampling2D,LeakyReLU,ZeroPadding2D,multiply
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16

from tensorflow.keras.layers import LeakyReLU

from keras.layers import Activation, SpatialDropout2D
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from keras.layers import Dense, Lambda
from keras.layers import concatenate, add
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Permute
from keras.optimizers import SGD
# from keras.preprocessing.image import ImageDataGenerator


AUTOTUNE = tf.data.experimental.AUTOTUNE
print(f"Tensorflow ver. {tf.__version__}")

2024-06-02 19:00:53.418757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-02 19:00:54.031402: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tensorflow ver. 2.16.1


/home/sukrit/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# Input image size that we are going to use
IMAGE_SIZE = 512
num_classes = 4
# Set batch size
batch = 6
# Set learning rate
LR = 1e-4
# Set epoch
EPOCHS = 100

In [3]:
# Set image and mask paths
image_path = "../../Datasets/IE-dataset-All-tts-augmented/images/"
mask_path = "../../Datasets/IE-dataset-All-tts-augmented/masks/"
training_data = "train/"
val_data = "validation/"
test_data = "test/"

In [4]:
# Function to create lists of paths for training, validation and testing data
def load_data():
  extensions = ["*.jpg", "*.tif", "*.tiff"]
  file_patterns = [os.path.join(image_path, training_data, ext) for ext in extensions]

  TRAIN_X = sorted(sum((glob(pattern) for pattern in file_patterns), []))
  # TRAIN_X = sorted(glob(os.path.join(image_path + training_data, "*.jpg")))
  train_x = TRAIN_X
# image_path
  TRAIN_Y = sorted(glob(os.path.join(mask_path + training_data, "*.png")))
  train_y = TRAIN_Y

  file_patterns = [os.path.join(image_path, val_data, ext) for ext in extensions]
  VALID_TEST_X = sorted(sum((glob(pattern) for pattern in file_patterns), []))
  # VALID_TEST_X = sorted(glob(os.path.join(image_path + val_data, "*.jpg")))
  valid_x = VALID_TEST_X

  VALID_TEST_Y = sorted(glob(os.path.join(mask_path + val_data, "*.png")))
  valid_y = VALID_TEST_Y

  file_patterns = [os.path.join(image_path, test_data, ext) for ext in extensions]
  TEST_X = sorted(sum((glob(pattern) for pattern in file_patterns), []))
  test_x = TEST_X
  TEST_Y = sorted(glob(os.path.join(mask_path + test_data, "*.png")))
  test_y = TEST_Y

  return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [5]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data()

print("Training data: ", len(train_x))
print("Validation data: ", len(valid_x))
print("Test data: ", len(test_x))

Training data:  308
Validation data:  16
Test data:  15


In [6]:
# Set image augmentation (to be called by tensorflow dataset later)
data_augmentation = tf.keras.Sequential([
  # layers.RandomFlip("horizontal_and_vertical"),
  # layers.RandomRotation(0.2),
  # layers.RandomZoom(.5, .2),
  # layers.RandomContrast(0.1)
  ],
  )

In [7]:
# Functions to read image and mask paths to output numpy arrays
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = np.expand_dims(x, axis=-1)
    return x

In [8]:
# Function to generate tensorflow dataset 
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.uint8])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 1])
    return x, y

num_threads = 4

def tf_dataset_train(x, y, batch=batch):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.repeat()
    dataset = dataset.map(tf_parse, num_parallel_calls=num_threads)
    dataset = dataset.batch(batch)
    # newdataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(1)
    return dataset
    


def tf_dataset_valid(x, y, batch=batch):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=500)
    dataset = dataset.repeat()
    dataset = dataset.map(tf_parse, num_parallel_calls=num_threads)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(1)
    return dataset

def tf_dataset_test(x, y):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


In [9]:
train_dataset = tf_dataset_train(train_x, train_y)
valid_dataset = tf_dataset_valid(valid_x, valid_y)
test_dataset = tf_dataset_test(test_x, test_y)

2024-06-02 19:00:55.751756: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-06-02 19:00:55.751778: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: ipsa-MS-7B98
2024-06-02 19:00:55.751784: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: ipsa-MS-7B98
2024-06-02 19:00:55.751848: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.171.4
2024-06-02 19:00:55.751869: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 470.239.6
2024-06-02 19:00:55.751874: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:251] kernel version 470.239.6 does not match DSO version 535.171.4 -- cannot find working devices in this configuration


In [10]:
# Function to load image/mask and convert colour space from BGR to RGB so that image/mask can be displayed correctly by matplotlib
def read_and_rgb(x):
    x = cv2.imread(x)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    return x

In [11]:
# Defining the Convolutional Block
def conv_block(inputs, num_filters):
	# Applying the sequence of Convolutional, Batch Normalization
	# and Activation Layers to the input tensor
	x = tf.keras.Sequential([
		# Convolutional Layer
		tf.keras.layers.Conv2D(num_filters, 3, padding='same'),
		# Batch Normalization Layer
		tf.keras.layers.BatchNormalization(),
		# Activation Layer
		tf.keras.layers.Activation('relu'),
		# Convolutional Layer
		tf.keras.layers.Conv2D(num_filters, 3, padding='same'),
		# Batch Normalization Layer
		tf.keras.layers.BatchNormalization(),
		# Activation Layer
		tf.keras.layers.Activation('relu')
	])(inputs)

	# Returning the output of the Convolutional Block
	return x


In [12]:
def unet_plus_plus_model(IMAGE_SIZE, num_classes=4, deep_supervision=False):
    # Encoding Path
    inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input_image")
    encoder = VGG16(input_tensor=inputs, include_top=False,weights="imagenet")
    
    skip_connection_names = ["block1_conv2", "block2_conv2", "block3_conv3", "block4_conv3","block5_conv3"]
    encoder_output = encoder.get_layer("block5_pool").output

    
    x = encoder_output #8
    x_skip_1 = encoder.get_layer(skip_connection_names[-1]).output # 16
    x_skip_2 = encoder.get_layer(skip_connection_names[-2]).output # 32
    x_skip_3 = encoder.get_layer(skip_connection_names[-3]).output # 64
    x_skip_4 = encoder.get_layer(skip_connection_names[-4]).output # 128
    x_skip_5 = encoder.get_layer(skip_connection_names[-5]).output # 256

    f4=conv_block(tf.keras.layers.concatenate(
        [x_skip_1, tf.keras.layers.UpSampling2D()(x)]), 512)
    
    x_31= conv_block(tf.keras.layers.concatenate(
        [x_skip_2, tf.keras.layers.UpSampling2D()(x_skip_1)]), 256)
    f3=conv_block(tf.keras.layers.concatenate(
        [x_31,x_skip_2, tf.keras.layers.UpSampling2D()(f4)]), 256)

    x_21= conv_block(tf.keras.layers.concatenate(
        [x_skip_3, tf.keras.layers.UpSampling2D()(x_skip_2)]), 128)
    x_22= conv_block(tf.keras.layers.concatenate(
        [x_21,x_skip_3, tf.keras.layers.UpSampling2D()(x_31)]), 128)
    f2=conv_block(tf.keras.layers.concatenate(
        [x_22,x_21,x_skip_3, tf.keras.layers.UpSampling2D()(f3)]), 128)

    x_11= conv_block(tf.keras.layers.concatenate(
        [x_skip_4, tf.keras.layers.UpSampling2D()(x_skip_3)]), 64)
    x_12= conv_block(tf.keras.layers.concatenate(
        [x_11,x_skip_4, tf.keras.layers.UpSampling2D()(x_21)]), 64)
    x_13= conv_block(tf.keras.layers.concatenate(
        [x_12,x_11,x_skip_4, tf.keras.layers.UpSampling2D()(x_22)]), 64)
    f1=conv_block(tf.keras.layers.concatenate(
        [x_13,x_12,x_11,x_skip_4, tf.keras.layers.UpSampling2D()(f2)]), 64)

    x_01= conv_block(tf.keras.layers.concatenate(
        [x_skip_5, tf.keras.layers.UpSampling2D()(x_skip_4)]), 32)
    x_02= conv_block(tf.keras.layers.concatenate(
        [x_01,x_skip_5, tf.keras.layers.UpSampling2D()(x_11)]), 32)
    x_03= conv_block(tf.keras.layers.concatenate(
        [x_02,x_01,x_skip_5, tf.keras.layers.UpSampling2D()(x_12)]), 32)
    x_04= conv_block(tf.keras.layers.concatenate(
        [x_03,x_02,x_01,x_skip_5, tf.keras.layers.UpSampling2D()(x_13)]), 32)
    f0=conv_block(tf.keras.layers.concatenate(
        [x_04,x_03,x_02,x_01,x_skip_5, tf.keras.layers.UpSampling2D()(f1)]), 32)
    
    
    if deep_supervision:
        outputs = [
            tf.keras.layers.Conv2D(num_classes, 1)(x_01),
            tf.keras.layers.Conv2D(num_classes, 1)(x_02),
            tf.keras.layers.Conv2D(num_classes, 1)(x_03),
            tf.keras.layers.Conv2D(num_classes, 1)(x_04),
            tf.keras.layers.Conv2D(num_classes, 1)(f0)
        ]
        # Concatenating the segmentation maps
        outputs = tf.keras.layers.concatenate(outputs, axis=0)
 
    # If deep supervision is disabled, then the model will output the final segmentation map
    # which is the segmentation map at the end of the decoding path
    else:
        outputs = tf.keras.layers.Conv2D(num_classes, 1)(f0)
 
    # Creating the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='Unet_plus_plus')
 
    # Returning the model
    return model
 
 
# Testing the model
    # Creating the model
model = unet_plus_plus_model(IMAGE_SIZE, num_classes, deep_supervision=False)

# Printing the model summary
model.summary()


Model: "Unet_plus_plus"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 512, 512,  │      1,792 │ input_image[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 512, 512,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 256, 256,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 256, 256,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 256, 256,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 128, 128,  │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 128, 128,  │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 128, 128,  │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 128, 128,  │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 64, 64,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 64, 64,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 64, 64,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 64, 64,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 32, 32,    │          0 │ block4_conv3[0][… │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv1        │ (None, 32, 32,    │  2,359,808 │ block4_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv2        │ (None, 32, 32,    │  2,359,808 │ block5_conv1[0][

 Total params: 32,621,316 (124.44 MB)

 Trainable params: 32,614,020 (124.41 MB)

 Non-trainable params: 7,296 (28.50 KB)

In [13]:
# Specify all the encoder layers in a list
encoder_layers = model.layers[0:-46]

In [14]:
# Freeze the encoder layers for transfer learning (so that weights are only Augmentedchanged for the decoder layers druing training)
for layer in encoder_layers:
    layer.trainable = False
    # print(layer, layer.trainable)

In [15]:
model.summary()

Model: "Unet_plus_plus"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 512, 512,  │      1,792 │ input_image[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 512, 512,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 256, 256,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 256, 256,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 256, 256,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 128, 128,  │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 128, 128,  │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 128, 128,  │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 128, 128,  │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 64, 64,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 64, 64,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 64, 64,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 64, 64,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 32, 32,    │          0 │ block4_conv3[0][… │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv1        │ (None, 32, 32,    │  2,359,808 │ block4_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv2        │ (None, 32, 32,    │  2,359,808 │ block5_conv1[0][

 Total params: 32,621,316 (124.44 MB)

 Trainable params: 17,899,332 (68.28 MB)

 Non-trainable params: 14,721,984 (56.16 MB)

In [16]:
# Functions for dice coefficient and dice loss 

import tensorflow.keras.backend as K

def dice_coef(y_true, y_pred, smooth=1e-7):
    y_true_f = K.flatten(K.one_hot(K.cast(y_true, 'int32'), num_classes=4)[Ellipsis,1:])
    y_pred_f = K.flatten(y_pred[...,1:])
    intersect = K.sum(y_true_f * y_pred_f, axis=-1)
    true_sum = K.sum(y_true_f , axis=-1)
    pred_sum = K.sum(y_pred_f, axis=-1)
    return K.mean((2. * intersect / (true_sum+pred_sum + smooth)))

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def mean_iou(y_true, y_pred, num_classes=4):
  smooth = 1e-7  
  y_true_f = K.flatten(K.one_hot(K.cast(y_true, 'int32'), num_classes=num_classes)[..., 1:])
  y_pred_f = K.flatten(y_pred[..., 1:])
  intersection = K.sum(K.cast(y_true_f * y_pred_f, dtype='float32'), axis=-1)
  set_sum = K.sum(K.cast(y_true_f + y_pred_f, dtype='float32'), axis=-1)
  iou = (intersection + smooth) / (set_sum -intersection+ smooth)
  mIoU = K.mean(iou)
  return mIoU

def mIoU_loss(y_true, y_pred):
    return 1 - mean_iou(y_true, y_pred)
   


In [17]:
# Set up loss function, optimiser and complile the model 
opt = tf.keras.optimizers.AdamW(LR)
metrics = [dice_coef,mean_iou]
model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)
#model.compile(optimizer=opt, loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [18]:
# Set up early stopping and reduce learning rate on plateau
early_stopping = EarlyStopping(
    monitor='mean_iou', 
    patience=4, 
    mode='max'
)

reduce_lr = ReduceLROnPlateau(
    monitor='mean_iou', 
    factor=0.8,   
    patience=3, 
    min_lr=0.0001,
)


In [19]:
# Set up checkpoint for saving model architecture and weights
checkpoint_filepath = 'checkpoint.model.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='mean_iou',
    mode='max',
    save_best_only=True)

In [ ]:
# Commencement of training
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0: 
    train_steps += 1
if len(valid_x) % batch != 0: 
    valid_steps += 1

history=model.fit(
    train_dataset,
    validation_data = valid_dataset,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=[model_checkpoint_callback,reduce_lr]
)


Epoch 1/100
 4/52 ━━━━━━━━━━━━━━━━━━━━ 33:02 41s/step - dice_coef: 0.7081 - loss: 0.2919 - mean_iou: 0.5622

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Training Loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.show()
plt.plot(history.history["mean_iou"])
plt.plot(history.history["val_mean_iou"])
plt.title("Training MIOU")
plt.ylabel("MIOU")
plt.xlabel("epoch")
plt.show()

In [ ]:
def mean_iouTEST(y_true, y_pred, num_classes=4):
  smooth = 1e-7  
  y_true_f = K.flatten(K.one_hot(K.cast(y_true, 'int32'), num_classes=num_classes)[..., 1:])
  y_pred_f = K.flatten(K.one_hot(K.cast(y_pred, 'int32'), num_classes=num_classes)[..., 1:])
  # y_pred_f = K.flatten(y_pred[..., 1:])
  intersection = K.sum(K.cast(y_true_f * y_pred_f, dtype='float32'), axis=-1)
  set_sum = K.sum(K.cast(y_true_f + y_pred_f, dtype='float32'), axis=-1)
  iou = (intersection + smooth) / (set_sum -intersection+ smooth)
  mIoU = K.mean(iou)
  return mIoU.numpy()

In [ ]:
def display_sample(sample_image,sample_mask_seperated,pred_mask_seperated):
    # plt.figure(figsize=(16, 16))
    title = ['bg','fat','cell','bone']
    # for i in range(len(display_list)):
    #     plt.subplot(1, len(display_list), i+1)
    #     plt.title(title[i])
    #     plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]),cmap='gray')
    #     plt.axis('off')
    # plt.show()

    fig,axs=plt.subplots(2,5,figsize=(30,12))
    axs[0][0].imshow(sample_image)
    axs[0][0].axis('off')
    axs[0][0].set_title('Original Image')
    for i in range(4):
        axs[0][i+1].imshow(sample_mask_seperated[i],cmap='gray')
        axs[0][i+1].axis('off')
        axs[0][i+1].set_title(f'Original {title[i]}')
    for i in range(4):
        axs[1][i+1].imshow(pred_mask_seperated[i],cmap='gray')
        axs[1][i+1].axis('off')
        axs[1][i+1].set_title(f'Predicted {title[i]}')
    plt.show()

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1) 
    pred_mask = tf.expand_dims(pred_mask, axis=-1)
    return pred_mask
    
def show_predictions(sample_image,sample_mask):
    one_img = sample_image[0][tf.newaxis, ...] 
    prediction = model.predict(one_img)
    pred_mask = create_mask(prediction)
    miou=mean_iouTEST(sample_mask[0],pred_mask)

    sample_mask_seperated=[(sample_mask[0]==i) for i in [0,1,2,3]]
    pred_mask_seperated=[(pred_mask[0]==i) for i in [0,1,2,3]]

    display_sample(sample_image[0], sample_mask_seperated,pred_mask_seperated)#sample_mask_seperated, pred_mask_seperated)
    
    fat_true=np.sum(sample_mask_seperated[1])
    cell_true=np.sum(sample_mask_seperated[2])
    fat_pred=np.sum(pred_mask_seperated[1])
    cell_pred=np.sum(pred_mask_seperated[2])
    cellularity_true=(cell_true*100)/(cell_true+fat_true)
    cellularity_pred=(cell_pred*100)/(cell_pred+fat_pred)
    cellularity_accuracy=(1-abs(cellularity_pred-cellularity_true))
    print(f"mIoU={miou*100:.3f}%\tcellulu true={cellularity_true:.3f}%\tcellulu pred={cellularity_pred:.3f}%\tDifference={cellularity_accuracy:.3f}%")
    return cellularity_accuracy,miou*100

In [ ]:
# for i in range(15):
def tf_dataset_test(x, y):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset=dataset.repeat()
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset
    
test_dataset = tf_dataset_test(test_x, test_y)

net_cellulu=[]
net_miou=[]
totalMasks=15

for image, mask in test_dataset.take(15):
        sample_image, sample_mask = image, mask
        cellulu,miou=show_predictions(sample_image, sample_mask)
        net_cellulu.append(cellulu)
        net_miou.append(miou)

In [ ]:
test_steps = len(test_x)//batch
if len(test_x) % batch != 0: 
    test_steps += 1

result = model.evaluate(test_dataset, steps=test_steps)
dict(zip(model.metrics_names, result))
print(f"Testing Cellularity Accuracy={np.mean(net_cellulu):.3f}%\tStandard Deviation: {np.std(net_cellulu):.3f}")
print(f"Testing mean_iou Standard Deviation: {np.std(net_miou):.3f}")

In [ ]:
# model.save('model_saves/VGG16_256.h5')  # Saves in the older HDF5 format
model.save(f"../../kerasModels/model_saves/Unet++{IMAGE_SIZE}.keras")  # Recommended format (uses ZIP archive)
model.save(f"../../kerasModels/model_saves/Unet++{IMAGE_SIZE}.h5")  # Recommended format (uses ZIP archive)